# Dashboard

### Pakete laden

In [1]:
import pandas as pd
import pyproj
from dash import dcc, html, Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.io as pio
import numpy as np

### Datein laden

In [2]:
# Daten laden
df = pd.read_csv("data/merged_df.csv",  sep=';', low_memory=False)
#"C:\Users\sha_r\OneDrive - FH Graubünden\FHGR_\4. Semester\Masterthesis\dahsboard\data\merged_df.csv"

px.set_mapbox_access_token("pk.eyJ1Ijoic2hhcm9uMDgxNSIsImEiOiJjbWM4dXo0cmYxb3Q5MmpzM2xuNjAwcWo0In0.JsHM9h7Y2Sak3JWE1vH8lw")
pio.renderers.default = 'browser'

# Wetterstationen und Unwetterarten ieren
stationen = df["Name"].dropna().unique()

df['time'] = pd.to_datetime(df['time'], errors='coerce')
time = df["time"].dropna().dt.strftime('%Y-%m-%d').unique()

unwetterarten = df["Unwetterart"].dropna().unique()

# Wettertypen (aus den numerischen Spalten)
wettertypen = ['Temp_Mittel', 'Temp_Abw', 'Niederschlag_Max_10min',
               'Niederschlag_Tag', 'Sonnenschein_h', 'Wind_kmh', 'Bodentemp_100cm']

df['time'] = pd.to_datetime(df['time'], errors='coerce')

In [34]:
df

,time,Station,Temp_Abw,Temp_Mittel,Niederschlag_Max_10min,Niederschlag_Tag,Sonnenschein_h,Wind_kmh,Bodentemp_100cm,Name,Koordinaten_GMS,Koordinaten_km,Hoehe_m,lon,lat,Unwetterart,Schweregrad
0,2000-01-01,ALT,1.3,1.5,0.0,0.0,1.2,4.0,NaN,Altdorf,8°37'/46°53',690180/193564,437.0,8.616667,46.883333,NaN,NaN
1,2000-01-02,ALT,2.0,2.2,0.0,0.0,3.8,4.3,NaN,Altdorf,8°37'/46°53',690180/193564,437.0,8.616667,46.883333,NaN,NaN
2,2000-01-03,ALT,-0.3,-0.1,0.0,0.0,0.6,5.0,NaN,Altdorf,8°37'/46°53',690180/193564,437.0,8.616667,46.883333,NaN,NaN
3,2000-01-04,ALT,-0.5,-0.3,0.0,0.0,2.6,4.7,NaN,Altdorf,8°37'/46°53',690180/193564,437.0,8.616667,46.883333,NaN,NaN
4,2000-01-05,ALT,2.6,2.8,0.0,0.0,0.0,3.2,NaN,Altdorf,8°37'/46°53',690180/193564,437.0,8.616667,46.883333,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256691,2022-05-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8°37'/46°53',689789/193142,NaN,6.634300,47.040900,Murgang,mittel
256692,2022-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8°37'/46°53',689789/193142,NaN,6.929500,45.866700,Rutschung,mittel
256693,2023-03-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8°37'/46°53',689789/193142,NaN,7.842900,46.785800,Rutschung,schwer
256694,2023-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8°37'/46°53',689789/193142,NaN,9.504600,47.336900,Rutschung,schwer


### App initialisieren

In [78]:
app = Dash(__name__)
app.title = "Wetter Dashboard Schweiz"

def create_tab_layout(tab_id):
    if tab_id == 1:
        return html.Div([
            html.Div([
                html.Div([
                    html.Label("Zeitraum:"),
                    dcc.DatePickerRange(
                        id=f'date-picker-{tab_id}',
                        start_date=min(df["time"]),
                        end_date=max(df["time"]),
                        display_format='YYYY-MM-DD',
                        className="date-picker",
                        with_portal=True,
                        min_date_allowed=min(df["time"]),
                        max_date_allowed=max(df["time"]),
                        reopen_calendar_on_clear=True
                    ),
                    html.Button("Zurücksetzen", id=f'reset-date-{tab_id}', n_clicks=0, className="reset-button"),
                ]),
                html.Label("Wettertyp:"),
                dcc.Dropdown(
                    id=f'wettertyp-{tab_id}',
                    options=[{'label': w, 'value': w} for w in wettertypen],
                    value="Temp_Mittel",
                    placeholder="Wettertyp wählen...",
                    className="dropdown"
                ),
                html.Label("Station:"),
                dcc.Dropdown(
                    id=f'station-{tab_id}',
                    options=[{'label': 'Alle Stationen', 'value': 'Alle'}] +
                            [{'label': s, 'value': s} for s in stationen],
                    placeholder="Station wählen...",
                    value="Alle",
                    className="dropdown"
                ),
                html.Label("Unwetterart:"),
                dcc.Dropdown(
                    id=f'unwetterart-{tab_id}',
                    options= [{'label': 'Alle Unwetterarten', 'value': 'Alle'}] + [{'label': w, 'value': w} for w in unwetterarten],
                    placeholder="Unwetterart wählen...",
                    value="Alle",
                    className="dropdown"
                )
            ], className="sidebar"),

            html.Div([
                html.Div([
                    dcc.Graph(id='plot-1-top-left', className="graph-box"),
                    dcc.Graph(id='plot-1-top-right', className="graph-box")  # neu: Scatter Mapbox oben rechts
                ], className="row-container"),

                html.Div([
                    dcc.Graph(id='plot-1-bottom-left', className="graph-box"),
                    dcc.Graph(id='plot-1-bottom-right', className="graph-box")  # neu: Scatter Mapbox unten rechts
                ], className="row-container")
            ], className="content-area")
        ], className="main-layout")

    elif tab_id == 2:
        return html.Div([
            html.Div([
                html.Label("Zeitraum:"),
                dcc.DatePickerRange(
                    id='date-picker-2',
                    start_date=min(df["time"]),
                    end_date=max(df["time"]),
                    display_format='YYYY-MM-DD',
                    with_portal=True,
                    min_date_allowed=min(df["time"]),
                    max_date_allowed=max(df["time"]),
                    reopen_calendar_on_clear=True,
                    style={"margin-bottom": "15px"}
                ),
                html.Label("Wettertyp:"),
                dcc.Dropdown(
                    id='wettertyp-2',
                    options=[{'label': w, 'value': w} for w in wettertypen],
                    value=wettertypen[0],
                    placeholder="Wettertyp wählen...",
                    className="dropdown"
                ),
            ], className="sidebar"),

            html.Div([
                dcc.Graph(
                    id='scatter-mapbox',
                    className="map-box",
                    style={"height": "500px", "width": "100%"}
                )
            ], className="content-area"),
        ], className="main-layout")

    elif tab_id == 3:
        return html.Div([
            html.Div([
                html.Label("Zeitraum:"),
                dcc.DatePickerRange(
                    id='date-picker-3',
                    start_date=min(df["time"]),
                    end_date=max(df["time"]),
                    display_format='YYYY-MM-DD',
                    with_portal=True,
                    min_date_allowed=min(df["time"]),
                    max_date_allowed=max(df["time"]),
                    reopen_calendar_on_clear=True,
                    style={"margin-bottom": "15px"}
                ),
                html.Label("Unwetterart:"),
                dcc.Dropdown(
                    id='unwetterart-3',
                    options=[{'label': 'Alle Unwetterarten', 'value': 'Alle'}] +
                            [{'label': w, 'value': w} for w in unwetterarten],
                    value='Alle',
                    placeholder="Unwetterart wählen...",
                    className="dropdown"
                ),
            ], className="sidebar"),

            html.Div([
                dcc.Graph(
                    id='scatter-mapbox-3',
                    className="map-box",
                    style={"height": "500px", "width": "100%"}
                    #style={"height": "600px"}
                )
            ], className="content-area"),
        ], className="main-layout")

    else:
        return html.Div([
            html.H3("Inhalt folgt",  style={"color": "white"}),
            html.P("Hier folgt eine interaktive Karte", style={"color": "white"})
        ], className="content-area")

app.layout = html.Div([
    html.H1("Wetter- und Naturereignisdaten – Schweiz", className="dashboard-title"),
    dcc.Tabs([
        dcc.Tab(label='Zeitverlauf', children=[create_tab_layout(1)]),
        dcc.Tab(label='Wetterstationen', children=[create_tab_layout(2)]),
        dcc.Tab(label='Naturereignis', children=[create_tab_layout(3)]),
        dcc.Tab(label='Über das Projekt', children=[
            html.Div([
                html.H3("Über das Projekt",  style={"color": "white"}),
                html.P("""
                    Dieses Dashboard visualisiert Wetter- und Naturereignisdaten aus der Schweiz.
                    Ziel ist es, Zusammenhänge und Trends zwischen Wetterdaten und Extremereignissen zu erkennen.
                """,  style={"color": "white"})
            ], className="content-area")
        ]),
    ])
])

# --- Callbacks Tab 1 ---

@app.callback(
    Output('date-picker-1', 'start_date'),
    Output('date-picker-1', 'end_date'),
    Input('reset-date-1', 'n_clicks'),
    prevent_initial_call=True
)
def reset_date(n_clicks):
    return min(df["time"]), max(df["time"])

@app.callback(
    Output('plot-1-top-left', 'figure'),
    Output('plot-1-bottom-left', 'figure'),
    Input('wettertyp-1', 'value'),
    Input('station-1', 'value'),
    Input('date-picker-1', 'start_date'),
    Input('date-picker-1', 'end_date'),
    Input('unwetterart-1', 'value'),
)
def update_tab1_plots(wettertyp, station, start_date, end_date, unwetterart):
    empty_fig = px.bar(title="Bitte Filter auswählen")
    empty_fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)'
    )

    if not wettertyp:
        return empty_fig, empty_fig

    dff = df.copy()
    dff['time'] = pd.to_datetime(dff['time'], errors='coerce')
    dff = dff[
        (dff['time'] >= pd.to_datetime(start_date)) &
        (dff['time'] <= pd.to_datetime(end_date))
    ]

    if station != "Alle":
        dff = dff[dff['Name'] == station]

    dff['Datum'] = dff['time'].dt.date
    df_agg = dff.groupby("Datum")[wettertyp].mean().reset_index()

    fig1 = px.line(
        df_agg,
        x="Datum", y=wettertyp,
        title=f"{wettertyp} pro Tag – {'alle Stationen' if station == 'Alle' else station}",
        color_discrete_sequence=["#2f2e88"]
    )
    fig1.update_layout(
        xaxis_title="Datum", yaxis_title=wettertyp,
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)'
    )

    dff_uw = df[df["Unwetterart"].notna()].copy()
    dff_uw['time'] = pd.to_datetime(dff_uw['time'], errors='coerce')
    dff_uw['Datum'] = dff_uw['time'].dt.date
    dff_uw = dff_uw[
        (dff_uw['time'] >= pd.to_datetime(start_date)) &
        (dff_uw['time'] <= pd.to_datetime(end_date))
    ]

    if unwetterart and unwetterart != "Alle":
        dff_uw = dff_uw[dff_uw['Unwetterart'] == unwetterart]

    if not dff_uw.empty:
        if unwetterart and unwetterart != "Alle":
            unwetter_counts = dff_uw.groupby("Datum").size().reset_index(name="Anzahl")
            fig2 = px.bar(unwetter_counts, x="Datum", y="Anzahl",
                          title=f"Unwetter: {unwetterart} pro Tag",
                          color_discrete_sequence=["#2f2e88"])
            fig2.update_layout(
                plot_bgcolor='rgba(0,0,0,0)',
                paper_bgcolor='rgba(0,0,0,0)'
            )
        else:
            unwetter_counts = dff_uw.groupby(["Datum", "Unwetterart"]).size().reset_index(name="Anzahl")
            farben = {
                "Rutschung": "#2f2e88",
                "Sturz": "#6b69d6",
                "Murgang": "#4b49a5",
                "Hochwasser": "#a9a9c5"
            }
            fig2 = px.bar(unwetter_counts, x="Datum", y="Anzahl", color="Unwetterart",
                          color_discrete_map=farben,
                          title="Unwetter pro Tag nach Art")
            fig2.update_layout(
                xaxis_title="Datum", yaxis_title="Anzahl",
                plot_bgcolor='rgba(0,0,0,0)',
                paper_bgcolor='rgba(0,0,0,0)',
                legend_title="Unwetterart"
            )
    else:
        fig2 = px.bar(title="Keine Unwetterdaten im Zeitraum vorhanden")
        fig2.update_layout(
            plot_bgcolor='rgba(0,0,0,0)',
            paper_bgcolor='rgba(0,0,0,0)'
        )

    return fig1, fig2


# --- Neuer Callback für die rechte Mapbox-Plots in Tab 1 ---

@app.callback(
    Output('plot-1-top-right', 'figure'),
    Output('plot-1-bottom-right', 'figure'),
    Input('date-picker-1', 'start_date'),
    Input('date-picker-1', 'end_date'),
    Input('wettertyp-1', 'value'),
    Input('unwetterart-1', 'value'),
)
def update_right_maps(start_date, end_date, wettertyp, unwetterart):

    def classify_change(delta, threshold):
        if delta < -threshold:
            return 'Abnahme'
        elif delta > threshold:
            return 'Zunahme'
        else:
            return 'Keine Veränderung'

    farben = {
        'Abnahme': 'blue',
        'Keine Veränderung': 'lightgray',
        'Zunahme': 'red'
    }

    # 1) DataFrame filtern
    dff = df.copy()
    dff['time'] = pd.to_datetime(dff['time'])
    dff = dff[(dff['time'] >= pd.to_datetime(start_date)) &
              (dff['time'] <= pd.to_datetime(end_date))]
    if dff.empty:
        empty = px.scatter_mapbox(title="Keine Daten im Zeitraum")
        return empty, empty

    # 2) In zwei Hälften splitten
    t0, t1 = dff['time'].min(), dff['time'].max()
    mid = t0 + (t1 - t0) / 2
    df1 = dff[dff['time'] <= mid]
    df2 = dff[dff['time'] >  mid]

    # 3) Oben rechts: Wettertyp
    if wettertyp:
        g1 = df1.groupby(['Name','lat','lon'])[wettertyp] \
               .mean().reset_index(name='wert1')
        g2 = df2.groupby(['Name','lat','lon'])[wettertyp] \
               .mean().reset_index(name='wert2')
        if g1.empty or g2.empty:
            fig_top = px.scatter_mapbox(title="Nicht genug Daten für Vergleich")
        else:
            diff = pd.merge(g1, g2, on=['Name','lat','lon'], how='inner')
            diff['delta'] = diff['wert2'] - diff['wert1']
            diff['delta_klasse'] = diff['delta'].apply(lambda x: classify_change(x, 0.1))

            fig_top = px.scatter_mapbox(
                diff, lat="lat", lon="lon",
                hover_name="Name",
                hover_data={"wert1":True, "wert2":True, "delta":True},
                color="delta_klasse",
                color_discrete_map=farben,
                zoom=7,
                title=f"Veränderung {wettertyp}"
            )

            fig_top.update_traces(marker=dict(size=20))  # feste Punktgröße
            
            fig_top.update_layout(
                mapbox_style="streets",
                legend=dict(orientation="h", y=-0.1, x=0.5, xanchor="center"),
                legend_title="Veränderung",
                margin={"r":0,"t":40,"l":0,"b":0}
            )
    else:
        fig_top = px.scatter_mapbox(title="Kein Wettertyp gewählt")

    # 4) Unten rechts: Unwetter
    uw = dff[dff['Unwetterart'].notna()].copy()
    if uw.empty:
        return fig_top, px.scatter_mapbox(title="Keine Unwetterdaten")

    # Name-Fallback für NaN
    uw['Name'] = uw['Name'].fillna(uw['lat'].round(4).astype(str) + ", " + uw['lon'].round(4).astype(str))

    uw1 = uw[uw['time'] <= mid]
    uw2 = uw[uw['time'] >  mid]

    if unwetterart and unwetterart != "Alle":
        uw1 = uw1[uw1['Unwetterart'] == unwetterart]
        uw2 = uw2[uw2['Unwetterart'] == unwetterart]

    g1_uw = uw1.groupby(['Name','lat','lon']).size().reset_index(name='anz1')
    g2_uw = uw2.groupby(['Name','lat','lon']).size().reset_index(name='anz2')

    if g1_uw.empty and g2_uw.empty:
        fig_bottom = px.scatter_mapbox(title="Keine Unwetterdaten zum Vergleich")
    else:
        dfuw = pd.merge(g1_uw, g2_uw, on=['Name','lat','lon'], how='outer').fillna(0)
        dfuw['delta'] = dfuw['anz2'] - dfuw['anz1']
        dfuw['delta_klasse'] = dfuw['delta'].apply(lambda x: classify_change(x, 1))

        fig_bottom = px.scatter_mapbox(
            dfuw, lat="lat", lon="lon",
            hover_name="Name",
            hover_data={"anz1":True, "anz2":True, "delta":True},
            color="delta_klasse",
            color_discrete_map=farben,
            zoom=7,
            title=f"Veränderung Unwetteranzahl ({unwetterart})"
        )

        fig_bottom.update_traces(marker=dict(size=20))  # feste Punktgröße
        
        fig_bottom.update_layout(
            mapbox_style="streets",
            legend=dict(orientation="h", y=-0.1, x=0.5, xanchor="center"),
            legend_title="Veränderung",
            margin={"r":0,"t":40,"l":0,"b":0}
        )

    return fig_top, fig_bottom


# --- Callback Tab 2 ---

@app.callback(
    Output('scatter-mapbox', 'figure'),
    Input('date-picker-2', 'start_date'),
    Input('date-picker-2', 'end_date'),
    Input('wettertyp-2', 'value'),
)
def update_scatter_map(start_date, end_date, wettertyp):
    if not start_date or not end_date or not wettertyp:
        return px.scatter_mapbox(title="Bitte Filter auswählen")

    dff = df.copy()
    dff = dff[(dff['time'] >= pd.to_datetime(start_date)) & (dff['time'] <= pd.to_datetime(end_date))]
    dff = dff.dropna(subset=[wettertyp])
    

    if dff.empty:
        return px.scatter_mapbox(title="Keine Daten im ausgewählten Zeitraum")

    # Mittelwerte pro Station
    grouped = dff.groupby(['Name', 'lat', 'lon'])[wettertyp].mean().reset_index()

    center_lat = grouped['lat'].mean() if not grouped['lat'].isnull().all() else 46.8
    center_lon = grouped['lon'].mean() if not grouped['lon'].isnull().all() else 8.2
    

    fig = px.scatter_mapbox(
        grouped,
        lat="lat",
        lon="lon",
        hover_name="Name",
        hover_data=[wettertyp],
        color=wettertyp,
        color_continuous_scale="Plasma",
        #range_color=[-max_abs, max_abs],
        zoom=7,
    )

    fig.update_traces(marker=dict(size=20))  # feste Punktgröße
    
    fig.update_layout(
        mapbox_style="streets",
        mapbox_center= {"lat": center_lat, "lon": center_lon},
        margin={"r":0, "t":40, "l":0, "b":0},
        title=f"Wetterparameter '{wettertyp}' von {start_date} bis {end_date}",
        uirevision='fixed'
    )
    return fig


# --- Callback Tab 3 ---

@app.callback(
    Output('scatter-mapbox-3', 'figure'),
    Input('date-picker-3', 'start_date'),
    Input('date-picker-3', 'end_date'),
    Input('unwetterart-3', 'value'),
)
def update_scatter_map_tab3(start_date, end_date, unwetterart):
    if not start_date or not end_date:
        return px.scatter_mapbox(title="Bitte Zeitraum wählen")

    dff = df.copy()
    dff['time'] = pd.to_datetime(dff['time'], errors='coerce')
    dff = dff[(dff['time'] >= pd.to_datetime(start_date)) & (dff['time'] <= pd.to_datetime(end_date))]
    dff = dff[dff['Unwetterart'].notna() & dff['lat'].notna() & dff['lon'].notna()]

    if unwetterart != "Alle":
        dff = dff[dff['Unwetterart'] == unwetterart]

    if dff.empty:
        return px.scatter_mapbox(title="Keine Unwetterdaten im ausgewählten Zeitraum und Filter")

    fig = px.scatter_mapbox(
        dff,
        lat="lat",
        lon="lon",
        hover_name="Name",
        hover_data=["Unwetterart", "Schweregrad", "time"],
        color="Schweregrad",
        color_discrete_map={"leicht": "green", "mittel": "orange", "schwer": "red"},
        title=f"Unwetterkarte: {unwetterart} vom {start_date} bis {end_date}",
        zoom= 7        
    )

    fig.update_traces(marker=dict(size=20))  # feste Punktgröße
    
    fig.update_layout(
        mapbox_style="streets",
        mapbox_center={"lat": dff['lat'].mean(), "lon": dff['lon'].mean()},
        margin={"r":0, "t":40, "l":0, "b":0}
    )
    return fig


if __name__ == '__main__':
    app.run_server(debug=True, port=8053)